In [1]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import glob
import pickle
import warnings
from scipy import misc
import scipy.ndimage
from skimage import feature
from PIL import Image
import os
import cv2
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.models import model_from_json
from keras.callbacks import EarlyStopping
from keras import backend as K

Using TensorFlow backend.


In [2]:
def get_train_and_test_data(procent, model_number, augmented_data_list):
    train_hollow_folder = "final_data_potato/data_" + str(procent) + "/model_" + str(model_number) + "/train/hollow/"
    train_metal_folder = "final_data_potato/data_" + str(procent) + "/model_" + str(model_number) + "/train/metal/"
    train_perfect_folder = "final_data_potato/data_" + str(procent) + "/model_" + str(model_number) + "/train/perfect/"
    
    test_hollow_folder = "final_data_potato/data_" + str(procent) + "/model_" + str(model_number) + "/test/hollow/"
    test_metal_folder = "final_data_potato/data_" + str(procent) + "/model_" + str(model_number) + "/test/metal/"
    test_perfect_folder = "final_data_potato/data_" + str(procent) + "/model_" + str(model_number) + "/test/perfect/"

    train_data_hollow = []
    train_data_metal = []
    train_data_perfect = []

    test_data_hollow = []
    test_data_metal = []
    test_data_perfect = []

    
    
    # Training data
    for filename in os.listdir(train_hollow_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints": 
            img = (cv2.imread(train_hollow_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            train_data_hollow.append(img)


    for filename in os.listdir(train_metal_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(train_metal_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            train_data_metal.append(img)


    for filename in os.listdir(train_perfect_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(train_perfect_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            train_data_perfect.append(img)
    
    
    # Test data
    for filename in os.listdir(test_hollow_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(test_hollow_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            test_data_hollow.append(img)


    for filename in os.listdir(test_metal_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(test_metal_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            test_data_metal.append(img)


    for filename in os.listdir(test_perfect_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(test_perfect_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            test_data_perfect.append(img)
            
            
    # Augmented training data
    for augmentation in augmented_data_list:
        if augmentation == "vae" or augmentation == "gan" or augmentation == "gan_2":
            augmented_hollow_folder = "final_data_potato/data_" + str(procent) + "/model_" + str(model_number) + "/" + augmentation + "/hollow/"    
            augmented_perfect_folder = "final_data_potato/data_" + str(procent) + "/model_" + str(model_number) + "/" + augmentation + "/perfect/"
            augmented_metal_folder = "final_data_potato/data_" + str(procent) + "/model_" + str(model_number) + "/" + augmentation + "/metal/"
        
            for filename in os.listdir(augmented_hollow_folder):
                if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
                    img = (cv2.imread(augmented_hollow_folder + filename, cv2.IMREAD_GRAYSCALE))
                    train_data_hollow.append(img)
            
            for filename in os.listdir(augmented_metal_folder):
                if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
                    img = (cv2.imread(augmented_metal_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
                    train_data_metal.append(img)

            for filename in os.listdir(augmented_perfect_folder):
                if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
                    img = (cv2.imread(augmented_perfect_folder + filename, cv2.IMREAD_GRAYSCALE))
                    train_data_perfect.append(img)
        else:    
            augmented_hollow_folder = "final_data_potato/data_" + str(procent) + "/model_" + str(model_number) + "/" + augmentation + "/hollow/"
            augmented_metal_folder = "final_data_potato/data_" + str(procent) + "/model_" + str(model_number) + "/" + augmentation + "/metal/"
            augmented_perfect_folder = "final_data_potato/data_" + str(procent) + "/model_" + str(model_number) + "/" + augmentation + "/perfect/"
        
            for filename in os.listdir(augmented_hollow_folder):
                if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
                    img = (cv2.imread(augmented_hollow_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
                    train_data_hollow.append(img)

            for filename in os.listdir(augmented_metal_folder):
                if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
                    img = (cv2.imread(augmented_metal_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
                    train_data_metal.append(img)


            for filename in os.listdir(augmented_perfect_folder):
                if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
                    img = (cv2.imread(augmented_perfect_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
                    train_data_perfect.append(img)

    # Convert lists to np array
    np_train_data_hollow = np.array(train_data_hollow)
    np_train_data_metal = np.array(train_data_metal)
    np_train_data_perfect = np.array(train_data_perfect)

    # Convert lists to np array
    np_test_data_hollow = np.array(test_data_hollow)
    np_test_data_metal = np.array(test_data_metal)
    np_test_data_perfect = np.array(test_data_perfect)

    # Data labels
    np_train_data_hollow_labels = np.full(len(np_train_data_hollow), 0)
    np_train_data_metal_labels = np.full(len(np_train_data_metal), 1)
    np_train_data_perfect_labels = np.full(len(np_train_data_perfect), 2)

    np_test_data_hollow_labels = np.full(len(np_test_data_hollow), 0)
    np_test_data_metal_labels = np.full(len(np_test_data_metal), 1)
    np_test_data_perfect_labels = np.full(len(np_test_data_perfect), 2)


    # Concatenate train and label data
    train_data =  np.concatenate((np_train_data_hollow, np_train_data_metal, np_train_data_perfect), axis=0) 
    train_labels = np.concatenate((np_train_data_hollow_labels, np_train_data_metal_labels, np_train_data_perfect_labels), axis=0)
 
    test_data = np.concatenate((np_test_data_hollow, np_test_data_metal, np_test_data_perfect), axis=0) 
    test_labels = np.concatenate((np_test_data_hollow_labels, np_test_data_metal_labels, np_test_data_perfect_labels), axis=0)

    train_data = train_data.reshape(train_data.shape[0], 128, 128, 1)
    test_data = test_data.reshape(test_data.shape[0], 128, 128, 1)

    return train_data, train_labels, test_data, test_labels

In [3]:
def run_model(train_data, train_labels, test_data, test_labels, patience):
    model = Sequential()
    #add model layers
    model.add(Conv2D(8, kernel_size=3, padding='same', strides=2, activation=tf.nn.relu, input_shape=(128,128,1)))
    model.add(Conv2D(8, kernel_size=3, padding='same', strides=2, activation=tf.nn.relu))
    model.add(Conv2D(8, kernel_size=3, padding='same', strides=2, activation=tf.nn.relu))
    #model.add(Conv2D(32, kernel_size=3, padding='same', strides=2, activation=tf.nn.relu))
    model.add(Flatten())
    model.add(Dense(1024, activation=tf.nn.relu))
    model.add(Dense(512, activation=tf.nn.relu))
    model.add(Dense(3, activation=tf.nn.softmax))

    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    #model.summary()
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience)


    history = model.fit(x=train_data,y=train_labels, validation_data=(test_data, test_labels), shuffle=True, batch_size=32, epochs=1000, verbose=0, callbacks=[es])
    history_acc = np.array(history.history['val_acc'])
    acc_mean = np.mean(history.history['val_acc'][-patience:])
    _, train_acc = model.evaluate(train_data, train_labels, verbose=0)
    _, test_acc = model.evaluate(test_data, test_labels, verbose=0)
    ynew = model.predict_classes(test_data)
    K.clear_session()
    return acc_mean, ynew, train_acc, test_acc


In [13]:
# Define augmentation
augmentaion_list = []

for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 0)
        #print(train_data.shape)
        for i in range(3):
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 40 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



no metal


Epoch 00070: early stopping
Run 0 with model 1 got train acc 0.5564516129032258 on test acc 0.53719007180742
Epoch 00075: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.39669421339823197
Epoch 00094: early stopping
Run 0 with model 3 got train acc 0.5564516129032258 on test acc 0.43801652843301947
Epoch 00080: early stopping
Run 0 with model 4 got train acc 0.5564516129032258 on test acc 0.35537190107274647
Epoch 00110: early stopping
Run 0 with model 5 got train acc 0.5564516129032258 on test acc 0.6859504102675382
Epoch 00090: early stopping
Run 0 with model 6 got train acc 0.5564516129032258 on test acc 0.35537189787084406
Epoch 00073: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.4958677690876417
Epoch 00090: early stopping
Run 0 with model 8 got train acc 0.5564516129032258 on test acc 0.6198347068029987
Epoch 00091: early stopping
Run 0 with model 9 got train acc 0.5564516129032258 on test acc 0.6198347033547961
Epoch 00088: ear

Epoch 00126: early stopping
Run 0 with model 7 got train acc 0.6680497925311203 on test acc 0.7037036948733859
Epoch 00236: early stopping
Run 0 with model 8 got train acc 0.6680497925311203 on test acc 0.8703703814082675
Epoch 00153: early stopping
Run 0 with model 9 got train acc 0.6680497925311203 on test acc 0.9074074184453046
Epoch 00164: early stopping
Run 0 with model 10 got train acc 0.6680497925311203 on test acc 0.8888889065495243
Model 0 got an acc on: 0.7962962985038757
Using 60 procent of data to training our model got an acc 0.8111111192791551
Hollow to hollow: 72.14285714285714
Hollow to metal: 7.857142857142857
Hollow to perfect: 20.0
Metal to hollow: 2.5
Metal to metal: 80.5
Metal to perfect: 17.0
Perfect to hollow: 4.5
Perfect to metal: 7.5
Perfect to perfect: 88.0
157.8839831352234
Epoch 00171: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8780487833953485
Epoch 00165: early stopping
Run 0 with model 2 got train acc 0.678030303030303 on test acc 0

In [13]:
augmentaion_list = ["rotation_180", "rotation_90_270", "flipping", "vae", "gan"]

print()
print()
print("metal")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 1)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 40 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



metal


Epoch 00070: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.7107437952490877
Epoch 00067: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.48760330603142416
Epoch 00098: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.6280991627165109
Epoch 00061: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.77685950216183
Epoch 00061: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.6776859385908143
Epoch 00070: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.8099173455198935
Epoch 00060: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.7851239561049406
Epoch 00066: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.7685950373815111
Epoch 00067: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.7190082578127049
Epoch 00069: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.7355371802306372
Epoch 00060: early stopping
Run

Epoch 00084: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.9148936385804034
Epoch 00099: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.734042578555168
Epoch 00084: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.6808510834866381
Epoch 00088: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.8085106712706546
Epoch 00089: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8723404369455703
Epoch 00078: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.8829787259406232
Epoch 00087: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.8510638602236484
Epoch 00083: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.8297872467243925
Epoch 00089: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.8829787360860947
Epoch 00079: early stopping
Run 2 with model 9 got train acc 1.0 on test acc 0.8404255547422044
Epoch 00083: early stopping
Run 0 with mo

Epoch 00125: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8518518540594313
Epoch 00106: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.851851847436693
Epoch 00116: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.8703703725779498
Epoch 00114: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.8888888999267861
Epoch 00159: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.9444444621050799
Epoch 00120: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.9259259369638231
Epoch 00113: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.907407425068043
Epoch 00117: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.9259259435865614
Epoch 00119: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.9444444444444444
Epoch 00109: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.9074074184453046
Epoch 00105: early stopping
Run 1 with mod

Epoch 00237: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 1.0
Epoch 00171: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00588: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 1.0
Epoch 00151: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00126: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00134: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00130: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00140: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00174: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.9259259104728699
Epoch 00136: early stopping
Run 2 with model 9 got train acc 1.0 on test acc 0.9259259104728699
Epoch 00128: early stopping
Run 0 with model 10 got train acc 1.0 on t

In [12]:
augmentaion_list = ["rotation_180", "rotation_90_270", "flipping", "vae", "gan"]

print()
print()
print("no metal")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data2(p, j + 1, augmentaion_list, 0)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 40 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



no metal


Epoch 00091: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.7107437996824911
Epoch 00108: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.7190082634776092
Epoch 00064: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.4132231382791661
Epoch 00072: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.6694214821847018
Epoch 00059: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.5289256178643092
Epoch 00071: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.6611570228229869
Epoch 00071: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.6363636422748408
Epoch 00074: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.7024793349021723
Epoch 00083: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.8429751967595629
Epoch 00073: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.7933884253186628
Epoch 00062: early stopping

Epoch 00112: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.8297872467243925
Epoch 00097: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.8404255369876293
Epoch 00101: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.7659574696358215
Epoch 00084: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.7234042781464597
Epoch 00088: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.7872340755259737
Epoch 00088: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8510638399327055
Epoch 00097: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.8085106484433437
Epoch 00093: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.8297872441880246
Epoch 00098: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.9042553292944077
Epoch 00115: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.8723404585046971
Epoch 00099: early stopping
Run 2 with m

Epoch 00124: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8148148236451326
Epoch 00130: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.8148148302678708
Epoch 00130: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.8518518430215342
Epoch 00112: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.8703703924461648
Epoch 00121: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.8888888933040477
Epoch 00136: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.9259259369638231
Epoch 00112: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.8333333509939688
Epoch 00118: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.9074074074074074
Epoch 00117: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.8888888888888888
Epoch 00115: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8703703747855293
Epoch 00123: early stopping
Run 1 with m

Epoch 00162: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 1.0
Epoch 00140: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00147: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00144: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00135: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00155: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00137: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00145: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00156: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.9259259104728699
Epoch 00159: early stopping
Run 2 with model 9 got train acc 1.0 on test acc 0.8518518805503845
Epoch 00140: early stopping
Run 0 with model 10 got tra

In [4]:
augmentaion_list = []



print("No agumentation")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 0)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)
    
    
    
    
augmentaion_list = ["rotation_180"]

print()
print()
print("Rotation 180")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 0)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)
    
    
augmentaion_list = ["flipping"]

print()
print()
print("Flipping")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 0)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)

No agumentation


Epoch 00060: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.5950413164028452
Epoch 00051: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.48760330874072616
Epoch 00052: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.5950413208362485
Epoch 00047: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.611570244978282
Epoch 00052: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.7107437908156844
Epoch 00055: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.6776859474576209
Epoch 00056: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.7355371890974439
Epoch 00056: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.6859504023859323
Epoch 00062: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.7520661137320779
Epoch 00050: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.6694214844014034
Epoch 00054: early sto

Epoch 00091: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.7446808738911406
Epoch 00163: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.8404255547422044
Epoch 00130: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.7872340552350308
Epoch 00111: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.755319173031665
Epoch 00085: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.7872340552350308
Epoch 00070: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.4893617351004418
Epoch 00121: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.7446808738911406
Epoch 00098: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.7872340729896058
Epoch 00088: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.7127659625195443
Epoch 00094: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.7553191628861935
Epoch 00095: early stopping
Run 2 with mo

Epoch 00121: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8518518673049079
Epoch 00177: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.8703703924461648
Epoch 00159: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.8703703747855293
Epoch 00148: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.9074074184453046
Epoch 00135: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.9074074184453046
Epoch 00136: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.9074074184453046
Epoch 00181: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.8518518562670108
Epoch 00108: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.8703703747855293
Epoch 00111: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.8333333487863894
Epoch 00137: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8703703747855293
Epoch 00149: early stopping
Run 1 with m

Epoch 00183: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00158: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00163: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00199: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00198: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00150: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00148: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00181: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00159: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.8518518805503845
Epoch 00179: early stopping
Run 2 with model 9 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00148: early stopping
Run 0 with m

Epoch 00073: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.7129629651705424
Epoch 00075: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.75
Epoch 00087: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.7222222266373811
Epoch 00054: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.574074078489233
Epoch 00065: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.6481481481481481
Epoch 00085: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.7407407407407407
Epoch 00080: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.7037037059112832
Epoch 00082: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.7129629629629629
Epoch 00098: early stopping
Run 1 with model 4 got train acc 1.0 on test acc 0.7592592592592593
Epoch 00092: early stopping
Run 2 with model 4 got train acc 1.0 on test acc 0.7222222222222222
Epoch 00066: early stopping
Run 0 with model 5 got trai

...

Epoch 00115: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8536585365853658
Epoch 00143: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.9024390243902439
Epoch 00147: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.8536585365853658
Epoch 00120: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.8048780487804879
Epoch 00135: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.8780487819415767
Epoch 00134: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.9024390243902439
Epoch 00106: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.8292682955904704
Epoch 00133: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.8048780516880315
Epoch 00134: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.8048780516880315
Epoch 00141: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8780487804878049
Epoch 00139: early stopping
Run 1 with m

Epoch 00161: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.8571429252624512
Epoch 00170: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.8571428656578064
Epoch 00132: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.785714328289032
Epoch 00134: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.9285714626312256
Epoch 00180: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.9285714626312256
Epoch 00162: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.9285714626312256
Epoch 00154: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.8571428656578064
Epoch 00176: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.9285714626312256
Run 2 with model 9 got train acc 1.0 on test acc 1.0
Epoch 00162: early stopping
Run 0 with model 10 got train acc 1.0 on test acc 0.785714328289032
Epoch 00155: early stopping
Run 1 with model 10 got train acc 1.0 on test acc 0.7857

Epoch 00088: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8723404559683292
Epoch 00077: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.8404255445967329
Epoch 00080: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.8936170428357226
Epoch 00099: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.8723404356773864
Epoch 00093: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.8829787335497268
Epoch 00086: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.8617021479505174
Epoch 00099: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.8936170415675386
Epoch 00081: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.8085106598569992
Epoch 00084: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.8404255623513079
Epoch 00091: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.7872340641123183
Epoch 00086: early stopping
Run 1 with m

Epoch 00149: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.8823529411764706
Epoch 00114: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.8970588235294118
Epoch 00090: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.8676470588235294
Epoch 00116: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.8823529411764706
Epoch 00089: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.8676470588235294
Epoch 00124: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8382352941176471
Epoch 00113: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.7647058823529411
Epoch 00095: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.8088235294117647
Epoch 00096: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.8235294117647058
Epoch 00129: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.8529411764705882
Epoch 00095: early stopping
Run 2 with m

Epoch 00116: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00130: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.8148148059844971
Epoch 00148: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00120: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.8148148059844971
Epoch 00169: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00129: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00132: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00125: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00118: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00135: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00134: early stopping
Run 1 with m

In [5]:
augmentaion_list = ["rotation_90_270"]

print()
print()
print("Rotation 90 270")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 1)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



Rotation 90 270


Epoch 00043: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.42975206217489953
Epoch 00084: early stopping
Run 1 with model 1 got train acc 0.9285714285714286 on test acc 0.6033057841387662
Epoch 00057: early stopping
Run 2 with model 1 got train acc 0.8333333333333334 on test acc 0.6363636309450323
Epoch 00071: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.7603305844236011
Epoch 00078: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.7107437908156844
Epoch 00062: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.6446280927697489
Epoch 00059: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.727272718898521
Epoch 00067: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.6859504100212381
Epoch 00127: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.8016528896063813
Epoch 00083: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8016528

Epoch 00070: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.7872340755259737
Epoch 00116: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.8404255648876758
Epoch 00087: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.8085106611251831
Epoch 00084: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.8085106598569992
Epoch 00089: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.7659574797812928
Epoch 00089: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.7872340641123183
Epoch 00077: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.76595745949035
Epoch 00084: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.8297872644789676
Epoch 00079: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.7872340653805022
Epoch 00076: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.7978723543755551
Epoch 00077: early stopping
Run 2 with mod

Epoch 00134: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8888888977192066
Epoch 00115: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.9074074118225662
Epoch 00104: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.8703703747855293
Epoch 00138: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.8148148236451326
Epoch 00144: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.8888888999267861
Epoch 00132: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.8703703747855293
Epoch 00100: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.8888888999267861
Epoch 00105: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.9074074184453046
Epoch 00107: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.8518518739276462
Epoch 00111: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8518518739276462
Epoch 00103: early stopping
Run 1 with m

Epoch 00246: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.9259259104728699
Epoch 00170: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00146: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00140: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00212: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00132: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00122: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00175: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.9259259104728699
Epoch 00164: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.9259259104728699
Epoch 00150: early stopping
Run 2 with model 9 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00154: early stopping
Run 0 with m

In [6]:
augmentaion_list = ["gaus_noise"]

print()
print()
print("Gaus Noise")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 1)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



Gaus Noise


Epoch 00032: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.33057851165779367
Epoch 00045: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.36363635181395476
Epoch 00044: early stopping
Run 2 with model 1 got train acc 0.9761904761904762 on test acc 0.404958671774746
Epoch 00049: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.5371900787038252
Epoch 00046: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.5123966912592738
Epoch 00041: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.561983468118778
Epoch 00051: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.5619834646705754
Epoch 00048: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.6033057752719595
Epoch 00047: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.6033057841387662
Epoch 00037: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.4380165257237174
Epoch 0004

Epoch 00067: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.6808510891934658
Epoch 00075: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.6808510879252819
Epoch 00076: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.6595744896442333
Epoch 00054: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.46808511526026625
Epoch 00067: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.6489361930400768
Epoch 00064: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.5957447024101906
Epoch 00062: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.6276595871499244
Epoch 00071: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.6276595871499244
Epoch 00058: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.680851068902523
Epoch 00063: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.6489361854309731
Epoch 00059: early stopping
Run 2 with m

Epoch 00099: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8333333377484922
Epoch 00095: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.72222223105254
Epoch 00093: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.7407407429483202
Epoch 00099: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.6666666622515078
Epoch 00108: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.7962963007114552
Epoch 00098: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.8333333554091277
Epoch 00102: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.7962963183720907
Epoch 00091: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.7777777998535721
Epoch 00091: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.7037037059112832
Epoch 00114: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8333333443712305
Epoch 00088: early stopping
Run 1 with mod

Epoch 00118: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00119: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00114: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00116: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.7407407760620117
Epoch 00120: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.8148148059844971
Epoch 00123: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00127: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00127: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.7777777910232544
Epoch 00140: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00127: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.8148148059844971
Epoch 00133: early stopping
Run 2 with m

In [7]:
augmentaion_list = ["scaling"]

print()
print()
print("Scaling")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 1)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



Scaling


Epoch 00042: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.438016527817269
Epoch 00042: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.4710743752392856
Epoch 00042: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.48760329839611843
Epoch 00044: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.570247926495292
Epoch 00050: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.6776859440094183
Epoch 00042: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.6115702415300794
Epoch 00049: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.6942148686440524
Epoch 00046: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.669421475780897
Epoch 00051: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.6611570193747843
Epoch 00049: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.7024793270205664
Epoch 00055: early stopping
Ru

Epoch 00069: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.7872340552350308
Epoch 00084: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.7659574696358215
Epoch 00068: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.7553191704952971
Epoch 00070: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.7553191704952971
Epoch 00072: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.7659574683676375
Epoch 00070: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.7340425747506162
Epoch 00072: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.7340425836279038
Epoch 00068: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.7234042654646203
Epoch 00070: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.7765957751172654
Epoch 00071: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.7340425671415126
Epoch 00065: early stopping
Run 2 with m

Epoch 00089: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8703703747855293
Epoch 00108: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.9074074118225662
Epoch 00092: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.8703703924461648
Epoch 00102: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.8333333554091277
Epoch 00107: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.8518518739276462
Epoch 00115: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.9259259435865614
Epoch 00090: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.8333333377484922
Epoch 00086: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.8148148192299737
Epoch 00090: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.7962963117493523
Epoch 00100: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8888888999267861
Epoch 00092: early stopping
Run 1 with m

Epoch 00120: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00127: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00120: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00123: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00128: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00121: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00117: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00170: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00130: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00126: early stopping
Run 2 with model 9 got train acc 1.0 on test acc 0.8148148059844971
Epoch 00122: early stopping
Run 0 with m

In [8]:
augmentaion_list = ["shifting"]

print()
print()
print("Shifting")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 1)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



Shifting


Epoch 00045: early stopping
Run 0 with model 1 got train acc 0.9761904761904762 on test acc 0.35537189787084406
Epoch 00040: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.404958674484048
Epoch 00054: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.3801652833449939
Epoch 00041: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.5123966868258705
Epoch 00045: early stopping
Run 1 with model 2 got train acc 0.9761904761904762 on test acc 0.6033057831535654
Epoch 00040: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.6363636299598315
Epoch 00050: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.7024793324391704
Epoch 00058: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.7190082615072076
Epoch 00052: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.669421475780897
Epoch 00053: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.603305781922064

Epoch 00068: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.7446808561365655
Epoch 00064: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.76595745949035
Epoch 00084: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.7553191704952971
Epoch 00068: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.7234042680009882
Epoch 00053: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.6702127786392861
Epoch 00063: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.680851086657098
Epoch 00062: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.7340425836279038
Epoch 00062: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.7127659701286478
Epoch 00056: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.6914893769203349
Epoch 00070: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.744680884036612
Epoch 00067: early stopping
Run 2 with model

Epoch 00117: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8518518673049079
Epoch 00097: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.8703703747855293
Epoch 00102: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.8333333487863894
Epoch 00118: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.8703703924461648
Epoch 00106: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.8703703924461648
Epoch 00107: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.907407425068043
Epoch 00095: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.8333333554091277
Epoch 00089: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.8148148192299737
Epoch 00092: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.7962963117493523
Epoch 00106: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8333333487863894
Epoch 00093: early stopping
Run 1 with mo

Epoch 00132: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00113: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00165: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00115: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00146: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00118: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00143: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00144: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00129: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00129: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00148: early stopping
Run 2 with m

In [9]:
augmentaion_list = ["vae"]

print()
print()
print("vae")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 1)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



vae


Epoch 00032: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.37190082324437856
Epoch 00032: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.30578512150394027
Epoch 00031: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.297520661403325
Epoch 00031: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.49586775677263245
Epoch 00032: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.4710743779485876
Epoch 00031: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.47107436834288036
Epoch 00032: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.5289256134309059
Epoch 00034: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.6033057821683647
Epoch 00039: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.5785123927534119
Epoch 00032: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.5206611503746884
Epoch 00034: early stopping
R

Epoch 00056: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.7234042794146436
Epoch 00052: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.6063829761870364
Epoch 00052: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.5744681003245902
Epoch 00051: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.5638298049886176
Epoch 00068: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.7765957751172654
Epoch 00070: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.7765957751172654
Epoch 00068: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.7553191793725845
Epoch 00054: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.5212766046219683
Epoch 00052: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.5638297948431461
Epoch 00084: early stopping
Run 2 with model 9 got train acc 1.0 on test acc 0.5851063937582868
Epoch 00053: early stopping
Run 0 with m

Epoch 00114: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8703703814082675
Epoch 00126: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.8148148192299737
Epoch 00092: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.8333333554091277
Epoch 00111: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.9259259259259259
Epoch 00100: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.8518518739276462
Epoch 00099: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.8333333487863894
Epoch 00093: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.8333333554091277
Epoch 00093: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.7962963183720907
Epoch 00090: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.7592592813350536
Epoch 00115: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8333333487863894
Epoch 00129: early stopping
Run 1 with m

Epoch 00167: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00122: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00141: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00189: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.9259259104728699
Epoch 00117: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00139: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.8148148059844971
Epoch 00125: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00268: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00137: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00134: early stopping
Run 2 with model 9 got train acc 1.0 on test acc 0.8518518805503845
Epoch 00116: early stopping
Run 0 with m

In [10]:
augmentaion_list = ["vae"]

print()
print()
print("vae")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 0)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



vae


Epoch 00034: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.4132231390180667
Epoch 00033: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.3801652885172978
Epoch 00058: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.636363635871036
Epoch 00040: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.6033057821683647
Epoch 00038: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.454545447156449
Epoch 00049: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.7107437952490877
Epoch 00043: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.5454545432378438
Epoch 00047: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.5619834644242752
Epoch 00044: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.6363636289746308
Epoch 00062: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.6942148809590616
Epoch 00042: early stopping
Run 1 

Epoch 00073: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.7021276747926752
Epoch 00080: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.7446808751593245
Epoch 00071: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.7340425861642715
Epoch 00068: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.744680884036612
Epoch 00075: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.7978723531073713
Epoch 00068: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.7872340729896058
Epoch 00108: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.765957477244925
Epoch 00069: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.6808510765116266
Epoch 00080: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.6808510891934658
Epoch 00075: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.6702127913211254
Epoch 00090: early stopping
Run 2 with mod

Epoch 00101: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8148148236451326
Epoch 00120: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.9074074294832017
Epoch 00103: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.8888889043419449
Epoch 00112: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.8333333487863894
Epoch 00108: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.7962963051266141
Epoch 00098: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.8518518562670108
Epoch 00098: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.8148148302678708
Epoch 00091: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.8333333554091277
Epoch 00098: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.8148148302678708
Epoch 00108: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8333333554091277
Epoch 00105: early stopping
Run 1 with m

Epoch 00118: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.9259259104728699
Epoch 00126: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.9259259104728699
Epoch 00135: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00133: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00139: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00123: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00176: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00169: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00139: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00163: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00159: early stopping
Run 2 with m

In [11]:
augmentaion_list = ["gan"]

print()
print()
print("vae")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 0)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



vae


Epoch 00054: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.4132231393875169
Epoch 00053: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.43801651907361244
Epoch 00063: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.43801652399961616
Epoch 00047: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.5537190141756673
Epoch 00052: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.4297520611896988
Epoch 00051: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.46280990553296303
Epoch 00046: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.6033057829072653
Epoch 00046: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.6115702405448787
Epoch 00077: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.7851239649717473
Epoch 00045: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.47933883706400215
Epoch 00052: early stopping


Epoch 00071: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.734042558264225
Epoch 00090: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.8723404559683292
Epoch 00078: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.7021276722563073
Epoch 00103: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.6702127887847575
Epoch 00081: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.7553191819089524
Epoch 00074: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.6808510968025695
Epoch 00086: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.7872340628441344
Epoch 00060: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.6702127786392861
Epoch 00063: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.4787234220098942
Epoch 00062: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.6063829952097953
Epoch 00086: early stopping
Run 2 with mo

Epoch 00117: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8333333487863894
Epoch 00104: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.8518518673049079
Epoch 00129: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.8333333554091277
Epoch 00120: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.9444444621050799
Epoch 00117: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.7777777799853572
Epoch 00111: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.72222223105254
Epoch 00103: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.8518518739276462
Epoch 00111: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.8148148368906092
Epoch 00108: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.8148148302678708
Epoch 00108: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.7962962874659786
Epoch 00107: early stopping
Run 1 with mod

Epoch 00137: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.9259259104728699
Epoch 00185: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00228: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00151: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00411: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00163: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00158: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00193: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00164: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.9259259104728699
Epoch 00186: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00166: early stopping
Run 2 with m

In [13]:
augmentaion_list = ["rotation_180", "rotation_90_270", "flipping"]

print()
print()
print("vae")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 1)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 40 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



vae


Epoch 00144: early stopping
Run 0 with model 1 got train acc 0.9795918367346939 on test acc 0.6363636368562368
Epoch 00087: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.6280991627165109
Epoch 00114: early stopping
Run 2 with model 1 got train acc 0.9489795918367347 on test acc 0.752066105850472
Epoch 00081: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.7355371890974439
Epoch 00111: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.8512396684362868
Epoch 00092: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.7190082659406111
Epoch 00127: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.7272727208689225
Epoch 00093: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.8264462800065348
Epoch 00123: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.8429752056263695
Epoch 00099: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8264462809917356
Epo

Epoch 00096: early stopping
Run 0 with model 6 got train acc 1.0 on test acc 0.8829787360860947
Epoch 00096: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.904255340708063
Epoch 00100: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.8936170428357226
Epoch 00078: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.7340425684096965
Epoch 00084: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.7553191616180095
Epoch 00100: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.7127659637877282
Epoch 00088: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8191489666066272
Epoch 00102: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.8191489666066272
Epoch 00084: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.8191489666066272
Epoch 00089: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.819148948852052
Epoch 00082: early stopping
Run 1 with mod

Epoch 00118: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8703703725779498
Epoch 00128: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.9074074228604635
Epoch 00122: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.9074073985770896
Epoch 00115: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.9074074184453046
Epoch 00165: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.9444444621050799
Epoch 00165: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.9074074184453046
Epoch 00142: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.9259259435865614
Epoch 00146: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.9444444444444444
Epoch 00127: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.9444444444444444
Epoch 00155: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8703703814082675
Epoch 00133: early stopping
Run 1 with m

Epoch 00589: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 1.0
Epoch 00139: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00142: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00849: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 1.0
Epoch 00150: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00136: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00138: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00210: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00160: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00179: early stopping
Run 2 with model 9 got train acc 1.0 on test acc 0.9259259104728699
Epoch 00136: early stopping
Run 0 with model 10 got train acc 1.0 on t

In [21]:
augmentaion_list = ["gan"]

print()
print()
print("vae")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 1)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



vae


Epoch 00031: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.4297520646379014
Epoch 00063: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.5123966927370749
Epoch 00031: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.3553719000875457
Epoch 00044: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.43801652399961616
Epoch 00052: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.6694214767660976
Epoch 00047: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.7355371836788398
Epoch 00055: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.7024793280057671
Epoch 00059: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.7685950393519125
Epoch 00055: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.528925622790313
Epoch 00031: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.40495867275994674
Epoch 00039: early stopping
Run

Epoch 00077: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.7340425760188001
Epoch 00076: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.7446808751593245
Epoch 00080: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.7234042882919312
Epoch 00064: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.7978723619846587
Epoch 00076: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.691489374383967
Epoch 00073: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.7659574670994536
Epoch 00067: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.7872340628441344
Epoch 00070: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.6489361943082607
Epoch 00051: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.5106383156269154
Epoch 00070: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.6914893667748634
Epoch 00079: early stopping
Run 2 with mo

Epoch 00188: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8703703747855293
Epoch 00214: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.8888889109646833
Epoch 00121: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.8518518673049079
Epoch 00119: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.8148148368906092
Epoch 00108: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.8888888933040477
Epoch 00141: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.8518518739276462
Epoch 00131: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.8148148302678708
Epoch 00116: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.7777777998535721
Epoch 00099: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.8703703747855293
Epoch 00121: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8703703924461648
Epoch 00100: early stopping
Run 1 with m

Epoch 00165: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00138: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00193: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00159: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00348: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 1.0
Epoch 00143: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00139: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00121: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00175: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00219: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00170: early stopping
Run 2 with model 9 got trai

In [20]:
augmentaion_list = ["rotation_180", "rotation_90_270", "flipping", "vae"]

print()
print()
print("vae")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data2(p, j + 1, augmentaion_list, 1)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 40 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



vae


Epoch 00108: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.7190082615072076
Epoch 00119: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.8347107428164522
Epoch 00111: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.8429752011929662
Epoch 00075: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.6859504058341349
Epoch 00069: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.6694214802143003
Epoch 00075: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.8429751977447636
Epoch 00062: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.7603305676751886
Epoch 00066: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.801652890591582
Epoch 00062: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.727272732198731
Epoch 00075: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8016528881285802
Epoch 00067: early stopping
Run 1 

Epoch 00107: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.8723404483592256
Epoch 00139: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.8617021517550691
Epoch 00086: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.734042578555168
Epoch 00084: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.7872340666486862
Epoch 00081: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.712765965055912
Epoch 00086: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8617021479505174
Epoch 00093: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.8510638310554179
Epoch 00100: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.8404255534740205
Epoch 00097: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.914893647457691
Epoch 00098: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.8829787449633821
Epoch 00088: early stopping
Run 2 with mode

Epoch 00128: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8888889043419449
Epoch 00107: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.9259259480017202
Epoch 00127: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.9074073985770896
Epoch 00140: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.8518518562670108
Epoch 00126: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.8888888933040477
Epoch 00169: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.9444444621050799
Epoch 00142: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.9259259435865614
Epoch 00122: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.9259259435865614
Epoch 00126: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.9074074184453046
Epoch 00190: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8333333554091277
Epoch 00114: early stopping
Run 1 with m

Epoch 00176: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 1.0
Epoch 00160: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00202: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00172: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 1.0
Epoch 00174: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00136: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00146: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00132: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00153: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00137: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00164: early stopping
Run 2 with model 9 got train acc 1.0 on te

In [23]:
augmentaion_list = ["rotation_180", "rotation_90_270", "flipping", "vae", "gan"]

print()
print()
print("vae")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data2(p, j + 1, augmentaion_list, 1)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 40 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



vae


Epoch 00071: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.7272727253023258
Epoch 00108: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.8264462809917356
Epoch 00078: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.867768590607919
Epoch 00069: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.7603305686603893
Epoch 00065: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.7107437996824911
Epoch 00083: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.8264462755731314
Epoch 00065: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.7768595036396311
Epoch 00061: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.826446279021334
Epoch 00064: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.7272727208689225
Epoch 00067: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8429752056263695
Epoch 00053: early stopping
Run 1 

Epoch 00103: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.904255340708063
Epoch 00092: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.6808510803161784
Epoch 00075: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.7553191819089524
Epoch 00089: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.8085106611251831
Epoch 00089: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8617021390732299
Epoch 00089: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.8510638602236484
Epoch 00087: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.8404255344512614
Epoch 00084: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.8829787436951982
Epoch 00092: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.9255319263072724
Epoch 00092: early stopping
Run 2 with model 9 got train acc 1.0 on test acc 0.904255321685304
Epoch 00088: early stopping
Run 0 with mod

Epoch 00114: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8888889043419449
Epoch 00119: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.8888888977192066
Epoch 00132: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.9259259480017202
Epoch 00120: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.8703703747855293
Epoch 00138: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.9074074184453046
Epoch 00128: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.9444444621050799
Epoch 00118: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.9259259259259259
Epoch 00116: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.9259259435865614
Epoch 00117: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.9444444444444444
Epoch 00112: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8888888999267861
Epoch 00187: early stopping
Run 1 with m

Epoch 00168: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 1.0
Epoch 00233: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00146: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00178: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 1.0
Epoch 00134: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00132: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00140: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00147: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00190: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.9259259104728699
Epoch 00149: early stopping
Run 2 with model 9 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00174: early stopping
Run 0 with model 10 got train acc 1.0 on t

In [9]:
augmentaion_list = []
procent = 20
model = 1



print("VAE all images ")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 0)
        print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)
    
    
'''
augmentaion_list = []
print()
print()
print("No aurgmentation images ")
print()
print()
for p in range(procent, procent + 10, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(model - 1, model):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 0)
        for i in range(10):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 10.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 10))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)
    
    
    
'''   
'''
augmentaion_list = ["final_vae"]
print()
print()
print("OLD vae ")
print()
print()
for p in range(procent, procent + 10, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(model - 1, model):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 0)
        for i in range(10):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 10.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 10))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)

    
    
augmentaion_list = ["final_vae"]
print()
print()
print("Good VAE images ")
print()
print()
for p in range(procent, procent + 10, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(model - 1, model):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 1)
        for i in range(10):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 10.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 10))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)
    
    

augmentaion_list = ["final_vae"]

print()
print()
print("Good VAE images no metal")
print()
print()
for p in range(procent, procent + 10, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(model - 1, model):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 0)
        for i in range(10):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 10.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 10))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)
'''        
    
    
    

    
    
    


VAE all images 


(134, 128, 128, 1)
Epoch 00058: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.5867768555633293
Epoch 00079: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.6694214836625029
Epoch 00053: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.5206611511135889
(134, 128, 128, 1)
Epoch 00038: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.5454545503805491
Epoch 00037: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.4049586658635415
Epoch 00040: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.6033057787201621
(134, 128, 128, 1)
Epoch 00062: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.7107437908156844
Epoch 00051: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.6198347033547961
Epoch 00042: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.578512384871806
(134, 128, 128, 1)
Epoch 00043: early stopping
Run 0 with mode

KeyboardInterrupt: 

In [49]:
'''
augmentaion_list = []

for p in range(10, 20, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(1):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 1)
        for i in range(20):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1
                    
                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1
                        
                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1
                
                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1
                    
                    
            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 20.0))
        
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 20))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)
'''

Epoch 00068: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.6198347092660006
Epoch 00077: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.5785123907830104
Epoch 00050: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.6363636348858352
Epoch 00060: early stopping
Run 3 with model 1 got train acc 1.0 on test acc 0.5619834661483765
Epoch 00062: early stopping
Run 4 with model 1 got train acc 1.0 on test acc 0.5702479316675959
Epoch 00052: early stopping
Run 5 with model 1 got train acc 1.0 on test acc 0.5619834651631758
Epoch 00061: early stopping
Run 6 with model 1 got train acc 0.9473684430122375 on test acc 0.6942148809590616
Epoch 00040: early stopping
Run 7 with model 1 got train acc 1.0 on test acc 0.4710743742540848
Epoch 00056: early stopping
Run 8 with model 1 got train acc 1.0 on test acc 0.5537190067866617
Epoch 00040: early stopping
Run 9 with model 1 got train acc 1.0 on test acc 0.5289256183569097
Epoch 00070: early stoppi

In [44]:
train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 0)
print(train_data.shape)

(14, 128, 128, 1)


In [8]:
start_path = "final_data_potato/"
def create_new_potato_folders(name):
    for i in range(10, 100, 10):
        data_path = start_path + "data_" + str(i) + "/"
        for j in range(1, 11, 1):
            model_path = data_path + "model_" + str(j) + "/"
            os.mkdir(model_path + name)
            os.mkdir(model_path + name + "/hollow")
            os.mkdir(model_path + name + "/metal")
            os.mkdir(model_path + name + "/perfect")

#create_new_potato_folders("rotation_180")
#create_new_potato_folders("rotation_90_270")
create_new_potato_folders("vae_2")

In [31]:
augmentaion_list = ["vae"]



print("VAE all images ")
print()
print()
for p in range(80, 90, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 0)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            while test_acc < 0.77:
                acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(10):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)

VAE all images 


Epoch 00142: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8518518209457397
Epoch 00176: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00178: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00129: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.7777777910232544
Epoch 00147: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.8148148059844971
Epoch 00169: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.8148148059844971
Epoch 00153: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00120: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00135: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00122: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.8888888955116272
Epoch 00118: early sto

In [20]:
augmentaion_list = ["vae_final_10000"]



print("VAE all images ")
print()
print()
for p in range(20, 30, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 0)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(10):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)

VAE all images 


Epoch 00070: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.6851851873927646
Epoch 00090: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.7685185229336774
Epoch 00070: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.6666666666666666
Epoch 00072: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.6574074085111972
Epoch 00053: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.5648148159186045
Epoch 00064: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.6388888888888888
Epoch 00059: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.6851851851851852
Epoch 00042: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.6296296296296297
Epoch 00060: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.6111111111111112
Epoch 00054: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.7037037037037037
Epoch 00043: early sto

In [4]:
augmentaion_list = ["vae"]



print("VAE all images with metal")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(0, 10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 1)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 10.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)

VAE all images 


Epoch 00031: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.27272727217309733
Epoch 00031: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.3471074355535271
Epoch 00032: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.24793388355862012
Epoch 00033: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.46280991341456895
Epoch 00034: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.4132231454218715
Epoch 00031: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.44628098779473424
Epoch 00033: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.42975206217489953
Epoch 00034: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.5041322373161631
Epoch 00033: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.43801652843301947
Epoch 00032: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.487603301844321
Epoch 00034: earl

Epoch 00053: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.5638297935749622
Epoch 00055: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.5744681015927741
Epoch 00052: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.45744682499702943
Epoch 00052: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.4148936411167713
Epoch 00052: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.4148936347758516
Epoch 00070: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.7127659777377514
Epoch 00063: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.7127659777377514
Epoch 00053: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.6808510841207301
Epoch 00053: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.5106383156269154
Epoch 00056: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.4787234194735263
Epoch 00053: early stopping
Run 2 with 

Epoch 00081: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.5740740873195507
Epoch 00083: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.3888888888888889
Epoch 00086: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.3888888888888889
Epoch 00104: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.8333333355409128
Epoch 00096: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.7777777910232544
Epoch 00094: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.759259268089577
Epoch 00100: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.7222222376752783
Epoch 00096: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.7407407495710585
Epoch 00092: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.7777777821929367
Epoch 00092: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.740740767231694
Epoch 00087: early stopping
Run 1 with mod

Epoch 00121: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.6666666865348816
Epoch 00125: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.7777777910232544
Epoch 00121: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.8148148059844971
Epoch 00121: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.703703761100769
Epoch 00116: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.8148148059844971
Epoch 00106: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.5925925970077515
Epoch 00107: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.6296296119689941
Epoch 00107: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.5925925970077515
Epoch 00112: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.7777777910232544
Epoch 00115: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.6666666865348816
Epoch 00114: early stopping
Run 2 with mo

In [10]:
augmentaion_list = ["rotation_180"]

print()
print()
print("Rotation 180")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 1)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)
    
    
augmentaion_list = ["flipping"]

print()
print()
print("Flipping")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list, 1)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 20 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



Rotation 180


Epoch 00080: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.6776859499206228
Epoch 00076: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.6280991686277153
Epoch 00074: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.7272727243171251
Epoch 00076: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.7438016440257553
Epoch 00089: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.801652882709976
Epoch 00072: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.7520661147172786
Epoch 00064: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.7438016499369597
Epoch 00061: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.7107437952490877
Epoch 00066: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.7438016455035564
Epoch 00064: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.7933884253186628
Epoch 00072: early stopp

Epoch 00129: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.9042553292944077
Epoch 00145: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.8617021580959888
Epoch 00104: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.7553191704952971
Epoch 00077: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.7765957751172654
Epoch 00073: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.6382978964359203
Epoch 00074: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.8191489463156842
Epoch 00067: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.7765957560945065
Epoch 00116: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.840425542060365
Epoch 00080: early stopping
Run 0 with model 9 got train acc 0.9146341463414634 on test acc 0.7340425633369608
Epoch 00091: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.776595766239978
Epoch 00071: early stopping

Epoch 00161: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.8703703924461648
Epoch 00153: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.8703703858234264
Epoch 00181: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.8888889043419449
Epoch 00148: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.8888888933040477
Epoch 00156: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.8888888933040477
Epoch 00150: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.9259259303410848
Epoch 00118: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.9444444444444444
Epoch 00168: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.8703703924461648
Epoch 00118: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.8703703814082675
Epoch 00190: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.9259259435865614
Epoch 00210: early stopping
Run 1 with m

Epoch 00220: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 1.0
Epoch 00467: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 1.0
Epoch 00149: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00164: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00150: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00138: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00160: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.9259259700775146
Epoch 00169: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00143: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.9259259104728699
Epoch 00155: early stopping
Run 2 with model 9 got train acc 1.0 on test acc 0.9629629850387573
Epoch 00226: early stopping
Run 0 with model 10 got train acc 1.0 on t

Epoch 00080: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.7129629629629629
Epoch 00079: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.8148148170223942
Epoch 00076: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.7962962985038757
Epoch 00059: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.6574074096149869
Epoch 00105: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.7407407429483202
Epoch 00079: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.7592592614668386
Epoch 00053: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.7129629651705424
Epoch 00055: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.731481483689061
Epoch 00100: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.7592592614668386
Epoch 00069: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.6666666688742461
Epoch 00070: early stopping
Run 1 with mo

Epoch 00109: early stopping
Run 1 with model 6 got train acc 1.0 on test acc 0.8765432113482629
Epoch 00098: early stopping
Run 2 with model 6 got train acc 1.0 on test acc 0.8271604967705998
Epoch 00103: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.8641975323359171
Epoch 00083: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.8641975323359171
Epoch 00090: early stopping
Run 2 with model 7 got train acc 1.0 on test acc 0.8641975323359171
Epoch 00108: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.814814829532011
Epoch 00107: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.8024691505196654
Epoch 00085: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.7654321017088713
Epoch 00114: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.8888889021343656
Epoch 00107: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.8395061750470856
Epoch 00089: early stopping
Run 2 with mo

Epoch 00177: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.9268292697464547
Epoch 00133: early stopping
Run 1 with model 1 got train acc 1.0 on test acc 0.8780487950255231
Epoch 00166: early stopping
Run 2 with model 1 got train acc 1.0 on test acc 0.9024390258440157
Epoch 00124: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.8292682955904704
Epoch 00110: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.8292682955904704
Epoch 00109: early stopping
Run 2 with model 2 got train acc 1.0 on test acc 0.8292682955904704
Epoch 00136: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.9024390272977876
Epoch 00127: early stopping
Run 1 with model 3 got train acc 1.0 on test acc 0.9024390258440157
Epoch 00124: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.9512195136488938
Epoch 00206: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.9024390243902439
Epoch 00185: early stopping
Run 1 with m

Epoch 00480: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 1.0
Epoch 00205: early stopping
Run 1 with model 7 got train acc 1.0 on test acc 0.9285714626312256
Run 2 with model 7 got train acc 1.0 on test acc 1.0
Epoch 00130: early stopping
Run 0 with model 8 got train acc 1.0 on test acc 0.9285714626312256
Epoch 00218: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.9285714626312256
Epoch 00153: early stopping
Run 2 with model 8 got train acc 1.0 on test acc 0.9285714626312256
Epoch 00226: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.7142857313156128
Epoch 00178: early stopping
Run 1 with model 9 got train acc 1.0 on test acc 0.9285714626312256
Epoch 00140: early stopping
Run 2 with model 9 got train acc 1.0 on test acc 0.8571428656578064
Epoch 00155: early stopping
Run 0 with model 10 got train acc 1.0 on test acc 0.785714328289032
Epoch 00124: early stopping
Run 1 with model 10 got train acc 1.0 on test acc 0.785714328289032
Ep

In [16]:
augmentaion_list = ["rotation_180", "rotation_90_270", "flipping", "vae"]

print()
print()
print("metal")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data3(p, j + 1, augmentaion_list, 1)
        #print(train_data.shape)
        for i in range(1):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 40 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



metal


Epoch 00052: early stopping
Run 0 with model 1 got train acc 0.7414448669201521 on test acc 0.39669421438343266
Epoch 00088: early stopping
Run 0 with model 2 got train acc 0.7908745247148289 on test acc 0.6198347077881994
Epoch 00075: early stopping
Run 0 with model 3 got train acc 0.7908745247148289 on test acc 0.7851239570901414
Epoch 00073: early stopping
Run 0 with model 4 got train acc 0.7908745247148289 on test acc 0.7685950403371133
Epoch 00080: early stopping
Run 0 with model 5 got train acc 0.7908745247148289 on test acc 0.7272727164355192
Epoch 00087: early stopping
Run 0 with model 6 got train acc 0.7908745247148289 on test acc 0.8016528861581786
Epoch 00062: early stopping
Run 0 with model 7 got train acc 0.7908745247148289 on test acc 0.5702479240322901
Epoch 00067: early stopping
Run 0 with model 8 got train acc 0.7908745247148289 on test acc 0.41322313556986406
Epoch 00085: early stopping
Run 0 with model 9 got train acc 0.7908745247148289 on test acc 0.595041

Epoch 00118: early stopping
Run 0 with model 6 got train acc 0.9008674101610905 on test acc 0.8888888933040477
Epoch 00110: early stopping
Run 0 with model 7 got train acc 0.8983890954151177 on test acc 0.8888889065495243
Epoch 00132: early stopping
Run 0 with model 8 got train acc 0.9008674101610905 on test acc 0.981481499142117
Epoch 00109: early stopping
Run 0 with model 9 got train acc 1.0 on test acc 0.7962963183720907
Epoch 00120: early stopping
Run 0 with model 10 got train acc 0.3593556381660471 on test acc 0.37037038140826756
Model 0 got an acc on: 0.3086419767803616
Using 60 procent of data to training our model got an acc 0.28333333693904644
Hollow to hollow: 77.85714285714286
Hollow to metal: 6.428571428571428
Hollow to perfect: 15.714285714285714
Metal to hollow: 0.5
Metal to metal: 84.0
Metal to perfect: 15.5
Perfect to hollow: 8.0
Perfect to metal: 1.0
Perfect to perfect: 91.0
314.9864459037781
Epoch 00167: early stopping
Run 0 with model 1 got train acc 0.90690032858707

In [21]:
augmentaion_list = ["vae"]

print()
print()
print("metal")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data3(p, j + 1, augmentaion_list, 0)
        #print(train_data.shape)
        for i in range(2):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 40 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 1.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 20))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



metal


Epoch 00113: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.7851239570901414
Epoch 00068: early stopping
Run 1 with model 1 got train acc 0.5564516129032258 on test acc 0.35537189971809546
Epoch 00098: early stopping
Run 0 with model 2 got train acc 1.0 on test acc 0.7438016430405546
Epoch 00076: early stopping
Run 1 with model 2 got train acc 0.5564516129032258 on test acc 0.5619834671335772
Epoch 00097: early stopping
Run 0 with model 3 got train acc 0.5483870967741935 on test acc 0.34710743210532446
Epoch 00093: early stopping
Run 1 with model 3 got train acc 0.5483870967741935 on test acc 0.3801652881478475
Epoch 00084: early stopping
Run 0 with model 4 got train acc 0.5564516129032258 on test acc 0.5041322230307523
Epoch 00093: early stopping
Run 1 with model 4 got train acc 0.5564516129032258 on test acc 0.4628099141534695
Epoch 00103: early stopping
Run 0 with model 5 got train acc 0.5564516129032258 on test acc 0.6363636334080341
Epoch 00124: early 

Epoch 00191: early stopping
Run 0 with model 3 got train acc 0.6391752577319587 on test acc 0.790123462677002
Epoch 00133: early stopping
Run 1 with model 3 got train acc 0.4793814432989691 on test acc 0.35802469140401594
Epoch 00121: early stopping
Run 0 with model 4 got train acc 1.0 on test acc 0.765432103180591
Epoch 00116: early stopping
Run 1 with model 4 got train acc 0.6391752577319587 on test acc 0.7654321024447311
Epoch 00115: early stopping
Run 0 with model 5 got train acc 0.4639175257731959 on test acc 0.37037037037037035
Epoch 00173: early stopping
Run 1 with model 5 got train acc 0.6391752577319587 on test acc 0.814814817758254
Epoch 00127: early stopping
Run 0 with model 6 got train acc 0.6391752577319587 on test acc 0.8271604952988801
Epoch 00111: early stopping
Run 1 with model 6 got train acc 0.8247422680412371 on test acc 0.37037037037037035
Epoch 00227: early stopping
Run 0 with model 7 got train acc 0.6391752577319587 on test acc 0.839506177254665
Epoch 00194: earl

Epoch 00151: early stopping
Run 0 with model 5 got train acc 0.678030303030303 on test acc 0.8536585511230841
Epoch 00192: early stopping
Run 1 with model 5 got train acc 0.678030303030303 on test acc 0.7804878077855925
Epoch 00203: early stopping
Run 0 with model 6 got train acc 0.678030303030303 on test acc 0.9512195121951219
Epoch 00152: early stopping
Run 1 with model 6 got train acc 0.678030303030303 on test acc 0.8536585394929095
Epoch 00164: early stopping
Run 0 with model 7 got train acc 0.678030303030303 on test acc 0.8780487833953485
Epoch 00166: early stopping
Run 1 with model 7 got train acc 0.678030303030303 on test acc 0.8048780516880315
Epoch 00162: early stopping
Run 0 with model 8 got train acc 0.678030303030303 on test acc 0.8048780545955752
Epoch 00162: early stopping
Run 1 with model 8 got train acc 1.0 on test acc 0.8536585424004531
Epoch 00151: early stopping
Run 0 with model 9 got train acc 0.678030303030303 on test acc 0.8048780516880315
Epoch 00116: early stopp

In [26]:
augmentaion_list = ["vae"]

print()
print()
print("metal")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data3(p, j + 1, augmentaion_list, 1)
        #print(train_data.shape)
        for i in range(1):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 40 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 1.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 20))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



metal


Epoch 00054: early stopping
Run 0 with model 1 got train acc 0.6927374301675978 on test acc 0.32231404638487443
Epoch 00068: early stopping
Run 0 with model 2 got train acc 0.6424581005586593 on test acc 0.36363636363636365
Epoch 00072: early stopping
Run 0 with model 3 got train acc 0.6927374301675978 on test acc 0.3553718968856433
Epoch 00054: early stopping
Run 0 with model 4 got train acc 0.6927374301675978 on test acc 0.4214876001038827
Epoch 00062: early stopping
Run 0 with model 5 got train acc 0.33519553072625696 on test acc 0.36363636363636365
Epoch 00051: early stopping
Run 0 with model 6 got train acc 0.6927374301675978 on test acc 0.5123966875647711
Epoch 00053: early stopping
Run 0 with model 7 got train acc 1.0 on test acc 0.41322313212166145
Epoch 00053: early stopping
Run 0 with model 8 got train acc 0.6871508379888268 on test acc 0.39669420847222825
Epoch 00061: early stopping
Run 0 with model 9 got train acc 0.6927374301675978 on test acc 0.3553718983634444


Epoch 00164: early stopping
Run 0 with model 6 got train acc 0.7507788161993769 on test acc 0.8333333554091277
Epoch 00123: early stopping
Run 0 with model 7 got train acc 0.7507788161993769 on test acc 0.8703703880310059
Epoch 00200: early stopping
Run 0 with model 8 got train acc 0.7507788161993769 on test acc 0.9074074118225662
Epoch 00138: early stopping
Run 0 with model 9 got train acc 0.3426791277258567 on test acc 0.37037038140826756
Epoch 00274: early stopping
Run 0 with model 10 got train acc 0.7507788161993769 on test acc 0.9259259435865614
Model 0 got an acc on: 0.7407407363255819
Using 60 procent of data to training our model got an acc 0.3944444509568038
Hollow to hollow: 70.71428571428572
Hollow to metal: 11.428571428571429
Hollow to perfect: 17.857142857142858
Metal to hollow: 2.5
Metal to metal: 82.0
Metal to perfect: 15.5
Perfect to hollow: 9.5
Perfect to metal: 9.0
Perfect to perfect: 81.5
192.1738531589508
Epoch 00159: early stopping
Run 0 with model 1 got train acc 

In [24]:
augmentaion_list = ["vae"]

print()
print()
print("metal")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data3(p, j + 1, augmentaion_list, 0)
        #print(train_data.shape)
        for i in range(2):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 40 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 1.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 20))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



metal


Epoch 00114: early stopping
Run 0 with model 1 got train acc 0.5564516129032258 on test acc 0.4876033052925236
Epoch 00085: early stopping
Run 1 with model 1 got train acc 0.5564516129032258 on test acc 0.4628099168627715
Epoch 00076: early stopping
Run 0 with model 2 got train acc 0.5564516129032258 on test acc 0.38016528777839725
Epoch 00072: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.4958677646542384
Epoch 00086: early stopping
Run 0 with model 3 got train acc 0.5564516129032258 on test acc 0.6528925501610622
Epoch 00111: early stopping
Run 1 with model 3 got train acc 0.5564516129032258 on test acc 0.6198346999065936
Epoch 00087: early stopping
Run 0 with model 4 got train acc 0.5564516129032258 on test acc 0.7024793321928702
Epoch 00097: early stopping
Run 1 with model 4 got train acc 0.5564516129032258 on test acc 0.5289256158939077
Epoch 00114: early stopping
Run 0 with model 5 got train acc 0.5564516129032258 on test acc 0.6859504033711331
Epoch

Epoch 00159: early stopping
Run 1 with model 2 got train acc 0.6391752577319587 on test acc 0.7777777792494974
Epoch 00177: early stopping
Run 0 with model 3 got train acc 0.6391752577319587 on test acc 0.8271604967705998
Epoch 00167: early stopping
Run 1 with model 3 got train acc 0.6391752577319587 on test acc 0.9012345708446738
Epoch 00138: early stopping
Run 0 with model 4 got train acc 0.6391752577319587 on test acc 0.7901234597335627
Epoch 00189: early stopping
Run 1 with model 4 got train acc 0.6391752577319587 on test acc 0.7901234604694225
Epoch 00140: early stopping
Run 0 with model 5 got train acc 0.6391752577319587 on test acc 0.7654321024447311
Epoch 00094: early stopping
Run 1 with model 5 got train acc 0.634020618556701 on test acc 0.5308642034177427
Epoch 00143: early stopping
Run 0 with model 6 got train acc 0.6391752577319587 on test acc 0.8271605085443567
Epoch 00172: early stopping
Run 1 with model 6 got train acc 0.6391752577319587 on test acc 0.8641975441096742
Ep

Epoch 00137: early stopping
Run 1 with model 4 got train acc 0.678030303030303 on test acc 0.9024390389279622
Epoch 00174: early stopping
Run 0 with model 5 got train acc 0.678030303030303 on test acc 0.8780487950255231
Epoch 00146: early stopping
Run 1 with model 5 got train acc 0.678030303030303 on test acc 0.8780487819415767
Epoch 00153: early stopping
Run 0 with model 6 got train acc 0.678030303030303 on test acc 0.7804878077855925
Epoch 00152: early stopping
Run 1 with model 6 got train acc 0.678030303030303 on test acc 0.9512195121951219
Epoch 00147: early stopping
Run 0 with model 7 got train acc 0.678030303030303 on test acc 0.8780487833953485


KeyboardInterrupt: 

In [27]:
augmentaion_list = ["gan_2"]

print()
print()
print("metal")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data3(p, j + 1, augmentaion_list, 1)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 40 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



metal


Epoch 00100: early stopping
Run 0 with model 1 got train acc 0.36312849162011174 on test acc 0.6611570238081876
Epoch 00084: early stopping
Run 1 with model 1 got train acc 0.3854748603351955 on test acc 0.5950413191121472
Epoch 00071: early stopping
Run 2 with model 1 got train acc 0.3854748603351955 on test acc 0.6033057856165673
Epoch 00108: early stopping
Run 0 with model 2 got train acc 0.3854748603351955 on test acc 0.6528925545944655
Epoch 00091: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.6363636289746308
Epoch 00066: early stopping
Run 2 with model 2 got train acc 0.6927374301675978 on test acc 0.5371900787038252
Epoch 00113: early stopping
Run 0 with model 3 got train acc 0.6927374301675978 on test acc 0.7603305730937926
Epoch 00090: early stopping
Run 1 with model 3 got train acc 0.6927374301675978 on test acc 0.7768595011766292
Epoch 00090: early stopping
Run 2 with model 3 got train acc 0.3854748603351955 on test acc 0.7272727208689225
Epoch

Epoch 00133: early stopping
Run 1 with model 3 got train acc 0.4491525423728814 on test acc 0.9148936170212766
Epoch 00153: early stopping
Run 2 with model 3 got train acc 1.0 on test acc 0.829787256869864
Epoch 00131: early stopping
Run 0 with model 4 got train acc 0.4491525423728814 on test acc 0.7872340641123183
Epoch 00102: early stopping
Run 1 with model 4 got train acc 0.4491525423728814 on test acc 0.8404255445967329
Epoch 00152: early stopping
Run 2 with model 4 got train acc 0.9449152542372882 on test acc 0.7978723531073713
Epoch 00093: early stopping
Run 0 with model 5 got train acc 0.7245762711864406 on test acc 0.8723404369455703
Epoch 00108: early stopping
Run 1 with model 5 got train acc 0.4491525423728814 on test acc 0.8404255445967329
Epoch 00122: early stopping
Run 2 with model 5 got train acc 0.7245762711864406 on test acc 0.8085106497115278
Epoch 00148: early stopping
Run 0 with model 6 got train acc 0.7245762711864406 on test acc 0.8191489589975235
Epoch 00111: earl

Epoch 00175: early stopping
Run 2 with model 5 got train acc 0.4863013698630137 on test acc 0.9411764705882353
Epoch 00151: early stopping
Run 0 with model 6 got train acc 0.7431506849315068 on test acc 0.9264705882352942
Epoch 00147: early stopping
Run 1 with model 6 got train acc 0.7431506849315068 on test acc 0.9264705882352942
Epoch 00158: early stopping
Run 2 with model 6 got train acc 0.7431506849315068 on test acc 0.9264705882352942
Epoch 00147: early stopping
Run 0 with model 7 got train acc 0.4863013698630137 on test acc 0.8529411764705882
Epoch 00130: early stopping
Run 1 with model 7 got train acc 0.4863013698630137 on test acc 0.8088235294117647
Epoch 00134: early stopping
Run 2 with model 7 got train acc 0.4863013698630137 on test acc 0.8676470588235294
Epoch 00138: early stopping
Run 0 with model 8 got train acc 0.7431506849315068 on test acc 0.8529411764705882
Epoch 00150: early stopping
Run 1 with model 8 got train acc 0.7431506849315068 on test acc 0.8529411764705882
E

Epoch 00133: early stopping
Run 2 with model 7 got train acc 0.5128939828080229 on test acc 0.9024390243902439
Epoch 00136: early stopping
Run 0 with model 8 got train acc 0.5128939828080229 on test acc 0.7804878106931361
Epoch 00175: early stopping
Run 1 with model 8 got train acc 0.5128939828080229 on test acc 0.8048780531418033
Epoch 00151: early stopping
Run 2 with model 8 got train acc 0.5128939828080229 on test acc 0.7804878106931361
Epoch 00174: early stopping
Run 0 with model 9 got train acc 0.5128939828080229 on test acc 0.7804878063318206
Epoch 00143: early stopping
Run 1 with model 9 got train acc 0.7564469914040115 on test acc 0.8048780516880315
Epoch 00143: early stopping
Run 2 with model 9 got train acc 0.7564469914040115 on test acc 0.7804878077855925
Epoch 00143: early stopping
Run 0 with model 10 got train acc 0.5128939828080229 on test acc 0.8536585380391377
Epoch 00148: early stopping
Run 1 with model 10 got train acc 0.7564469914040115 on test acc 0.8292682941366987

Epoch 00195: early stopping
Run 1 with model 10 got train acc 0.7660098522167488 on test acc 0.785714328289032
Epoch 00161: early stopping
Run 2 with model 10 got train acc 0.45320197044334976 on test acc 0.5714285969734192
Model 0 got an acc on: 0.809523860613505
Using 90 procent of data to training our model got an acc 0.8500000238418579
Hollow to hollow: 76.66666666666667
Hollow to metal: 4.166666666666666
Hollow to perfect: 19.166666666666668
Metal to hollow: 4.666666666666667
Metal to metal: 82.0
Metal to perfect: 13.333333333333334
Perfect to hollow: 1.3333333333333335
Perfect to metal: 4.0
Perfect to perfect: 94.66666666666667
924.9152789115906


In [28]:
augmentaion_list = ["vae"]

print()
print()
print("metal")
print()
print()
for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data3(p, j + 1, augmentaion_list, 1)
        #print(train_data.shape)
        for i in range(3):
            #train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1)
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 40 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)



metal


Epoch 00052: early stopping
Run 0 with model 1 got train acc 1.0 on test acc 0.4380165180884117
Epoch 00062: early stopping
Run 1 with model 1 got train acc 0.6703910614525139 on test acc 0.305785123782217
Epoch 00059: early stopping
Run 2 with model 1 got train acc 0.6703910614525139 on test acc 0.40495867275994674
Epoch 00052: early stopping
Run 0 with model 2 got train acc 0.6927374301675978 on test acc 0.42975205774149616
Epoch 00077: early stopping
Run 1 with model 2 got train acc 1.0 on test acc 0.570247926495292
Epoch 00051: early stopping
Run 2 with model 2 got train acc 0.3854748603351955 on test acc 0.35537189934864516
Epoch 00051: early stopping
Run 0 with model 3 got train acc 1.0 on test acc 0.4049586762081493
Epoch 00070: early stopping
Run 1 with model 3 got train acc 0.3743016759776536 on test acc 0.371900826446281
Epoch 00065: early stopping
Run 2 with model 3 got train acc 0.6927374301675978 on test acc 0.38842975157351534
Epoch 00053: early stopping
Run 0 w

Epoch 00115: early stopping
Run 1 with model 3 got train acc 0.7245762711864406 on test acc 0.5851063994651146
Epoch 00125: early stopping
Run 2 with model 3 got train acc 0.7245762711864406 on test acc 0.7765957573626904
Epoch 00101: early stopping
Run 0 with model 4 got train acc 0.7245762711864406 on test acc 0.6063830116961865
Epoch 00103: early stopping
Run 1 with model 4 got train acc 0.4491525423728814 on test acc 0.7872340666486862
Epoch 00098: early stopping
Run 2 with model 4 got train acc 0.7245762711864406 on test acc 0.6170212930821358
Epoch 00084: early stopping
Run 0 with model 5 got train acc 0.7245762711864406 on test acc 0.638297881217713
Epoch 00107: early stopping
Run 1 with model 5 got train acc 0.7245762711864406 on test acc 0.563829798013606
Epoch 00332: early stopping
Run 2 with model 5 got train acc 0.7245762711864406 on test acc 0.8085106433706081
Epoch 00160: early stopping
Run 0 with model 6 got train acc 0.4491525423728814 on test acc 0.8191489589975235
Epo

Epoch 00126: early stopping
Run 2 with model 5 got train acc 0.7431506849315068 on test acc 0.8529411764705882
Epoch 00252: early stopping
Run 0 with model 6 got train acc 0.7431506849315068 on test acc 0.8823529411764706
Epoch 00131: early stopping
Run 1 with model 6 got train acc 0.4863013698630137 on test acc 0.7941176470588235
Epoch 00124: early stopping
Run 2 with model 6 got train acc 0.7431506849315068 on test acc 0.7647058823529411
Epoch 00117: early stopping
Run 0 with model 7 got train acc 0.7431506849315068 on test acc 0.7941176470588235
Epoch 00129: early stopping
Run 1 with model 7 got train acc 0.4863013698630137 on test acc 0.7205882352941176
Epoch 00114: early stopping
Run 2 with model 7 got train acc 0.7431506849315068 on test acc 0.7941176470588235
Epoch 00169: early stopping
Run 0 with model 8 got train acc 0.7431506849315068 on test acc 0.6323529411764706
Epoch 00147: early stopping
Run 1 with model 8 got train acc 0.7431506849315068 on test acc 0.8235294117647058
E

Epoch 00258: early stopping
Run 0 with model 8 got train acc 0.7564469914040115 on test acc 0.7804878106931361
Epoch 00171: early stopping
Run 1 with model 8 got train acc 0.7564469914040115 on test acc 0.8048780495073737
Epoch 00185: early stopping
Run 2 with model 8 got train acc 0.7564469914040115 on test acc 0.8780487848491203
Epoch 00147: early stopping
Run 0 with model 9 got train acc 0.34097421203438394 on test acc 0.36585365853658536
Epoch 00260: early stopping
Run 1 with model 9 got train acc 0.7564469914040115 on test acc 0.8536585380391377
Epoch 00158: early stopping
Run 2 with model 9 got train acc 1.0 on test acc 0.9024390258440157
Epoch 00181: early stopping
Run 0 with model 10 got train acc 0.7564469914040115 on test acc 0.8048780502342596
Epoch 00141: early stopping
Run 1 with model 10 got train acc 0.3151862464183381 on test acc 0.2682926829268293
Epoch 00156: early stopping
Run 2 with model 10 got train acc 0.7564469914040115 on test acc 0.6829268321758364
Model 0 got

Epoch 00173: early stopping
Run 2 with model 10 got train acc 0.7660098522167488 on test acc 0.785714328289032
Model 0 got an acc on: 0.6904762387275696
Using 90 procent of data to training our model got an acc 0.7690476447343826
Hollow to hollow: 63.33333333333333
Hollow to metal: 15.0
Hollow to perfect: 21.666666666666668
Metal to hollow: 4.666666666666667
Metal to metal: 78.66666666666666
Metal to perfect: 16.666666666666664
Perfect to hollow: 2.0
Perfect to metal: 12.0
Perfect to perfect: 86.0
860.6631543636322
